In [1]:
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import urllib.request
from seleniumbase import Driver

import os
from dotenv import load_dotenv

In [34]:
ticket_info = pd.read_csv('data/ticket_site_concert_info_preprocessed.csv', encoding='UTF-8')
ticket_info

,공연명,좌석,가격,양도 불가능,예매처
0,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT］ IN SEOUL,선예매,132000,1.0,인터파크
1,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT］ IN SEOUL,일반예매,143000,1.0,인터파크
2,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT:THE HIDDEN S...,전석,154000,1.0,인터파크
3,2024 온앤오프(ONF) CONCERT ［SPOTLIGHT:THE HIDDEN S...,시야제한석,132000,1.0,인터파크
4,2024 ZEROBASEONE THE FIRST TOUR ［TIMELESS WORL...,VIP석,198000,1.0,인터파크
...,...,...,...,...,...
146,2024 tripleS FAN-CON < Girls Never Stop >,전석,110000,1.0,티켓링크
147,2024 ARTMS World Tour < Moonshot > in Seoul,전석,132000,1.0,티켓링크
148,뮤직플래닛 2024_ LUCY X N.Flying,R석,121000,1.0,티켓링크
149,뮤직플래닛 2024_ LUCY X N.Flying,S석,110000,1.0,티켓링크


In [36]:
keywords = list(set(concert['공연명']))
keywords

['뮤직플래닛, LUCY X N.Flying',
 'NCT127 3RD TOUR, THE UNITY: NEO CITY SEOUL',
 'aespa LIVE TOUR, SYNK: PARALLEL LINE',
 'ATEEZ WORLD TOUR: TOWARDS THE LIGHT: WILL TO POWER [서울]',
 'EVNNE FAN-CONCERT, SQUAD:R ASIA TOUR [서울]',
 'CRAVITY FAN-CON: LUVITY GAMES',
 'ONEWE GOLDEN FESTA: ONEderland',
 '권은비 세 번째 콘서트: QUEEN',
 'fromis_9 concert: FROM NOW.',
 'Flight1112: 기욱(ONEWE)편, 기욱의 음악으로 떠나는 여행!',
 'SF9 YOO TAE YANG FAN-CON: Double Casting: No. YTY',
 'VIXX LIVE FANTASIA: CONTINUUM',
 'ONEWE 3rd Live Concert, O! NEW E! volution Ⅲ',
 'NCT WISH: SCHOOL of WISH [전주]',
 'NCT DREAM WORLD TOUR, THE DREAM SHOW 3: DREAM ( ) SCAPE',
 'HIGHLIGHT LIVE: LIGHTS GO ON, AGAIN',
 'G, IDLE WORLD TOUR: iDOL [서울]',
 'Xdinary Heroes Concert: Closed beta: v6.1',
 '어썸스테이지, VANNER x M.O.N.T',
 '어썸스테이지, DAY6 & 오월오일 [부산]',
 'B1A4 13th Anniversary FAN CONCERT: 13ANA=DAY',
 'Xdinary Heroes Concert, Closed ♭eta: v6.4',
 'FTISLAND CONCERT, FTSODE',
 'Stray Kids World Tour: dominATE [서울]',
 'SEVENTEEN TOUR, FOLLOW AGAIN [서울]

In [37]:
load_dotenv('GOOGLE_login.env')

USER_ID = os.environ.get('GOOGLLE_USER_ID')
USER_PASSWORD = os.environ.get('GOOGLE_USER_PASSWORD')
USER_NUM = os.environ.get('GOOGLE_NUM')

In [38]:
x_pass_ticket_tweet = pd.DataFrame({'공연명': [],
                                   '트윗 내용' : [],
                                   '암표' : [],
                                   '양도 완료' : []})

In [40]:
# 로그인 세션
options = Options()
service = Service(excutable_path=ChromeDriverManager().install())
driver = Driver(browser='chrome', uc=True)

driver.implicitly_wait(10)
driver.maximize_window()

driver.get('https://x.com/home')
login = driver.find_element(By.XPATH, '/html/body/div/div/div/div[2]/main/div/div/div[1]/div[1]/div/div[3]/div[4]/a')
login.click()

driver.implicitly_wait(5)
driver.switch_to.window(driver.window_handles[-1]) 



id = driver.find_element(By.XPATH, '/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[4]/label/div/div[2]/div/input')
id.send_keys(USER_ID)
id.send_keys(Keys.RETURN)

# 비정상 로그인 잡혔을 때 2차 인증 처리
try:
    pw = driver.find_element(By.XPATH, '/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input')
except:
    confirm = driver.find_element(By.XPATH, '/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div[2]/label/div/div[2]/div/input')
    confirm.send_keys(USER_NUM)
    confirm.send_keys(Keys.RETURN)

pw = driver.find_element(By.XPATH, '/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input')    
pw.send_keys(USER_PASSWORD)
pw.send_keys(Keys.RETURN)

driver.implicitly_wait(5)
driver.switch_to.window(driver.window_handles[-1])
# if(len(driver.window_handles) != 1):
#     print(driver.window_handles)
#     google_login_again = driver.find_element(By.XPATH, '/html/body/div/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[2]/div/iframe')
#     google_login_again.click()

#키워드 순회


while(len(driver.window_handles) != 1):
    continue

driver.get('https://x.com/explore')
driver.switch_to.window(driver.window_handles[-1])
driver.implicitly_wait(10)

for keyword in keywords[:1]:
    search_word = keyword + ' "양도"'
    search = driver.find_elements(By.XPATH, '//input')[0]
    search.click()
    search.send_keys(search_word)
    search.send_keys(Keys.RETURN)
    
    
    
    




In [ ]:
# 양도 -> 서치 : 댈티 및 양도 글 총 개수만 세기
# 양도해요 양도합니다 -> 서치
# 

In [30]:
"가나다" + " 라마"

'가나다 라마'